# Run Fact Checking Benchmark
This notebook contains the code to run the CalibraGPT fact checking benchmark. Fact checking is accomplished by probing whether factual statements are predicted at a higher probability compared to paired counterfactual statements. We will be using the [CalibraGPT/Fact_Checking](https://huggingface.co/datasets/CalibraGPT/Fact_Checking) dataset. See the Repo's [README](https://github.com/daniel-furman/Capstone#model-families-tested) for compatible models and more information.

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/fact_checking_notebooks/fact-checking-full-benchmark.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dependencies

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt
#!pip install -r /content/Capstone/requirements_llama.txt

## Imports

In [ ]:
import os
import glob
import torch
from argparse import Namespace
from transformers import set_seed
from datasets import load_dataset

os.chdir("/content/Capstone/src/fact_checking_scripts")
from compare_models import compare_models

## Configure Args

In [ ]:
# args config for running the benchmark
args = Namespace(
    model="/content/drive/MyDrive/Colab Files/llama/LLaMA/int8/llama-13b",
    language="de",
)

args

## Setup

In [ ]:
# ensure GPU access
if not torch.cuda.is_available():
    raise Exception("Change runtime type to include a GPU.")

In [ ]:
# set warning level
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

## Run

In [ ]:
# run the fact checking benchmark
print("Running the fact_checking benchmark...")

set_seed(42)

# change the below to work on a list of ("english", "en") pairs
# load in the dataset corresponding to the input language
supported_languages = [
    ("english", "en"),
    ("french", "fr"),
    ("spanish", "es"),
    ("chinese", "zh"),
    ("japanese", "ja"),
    ("german", "de"),
]

dataset_bool = False
for lang_arr in supported_languages:
    if (args.language.lower() == lang_arr[0]) or (args.language.lower() == lang_arr[1]):
        dataset = load_dataset(
            "CalibraGPT/Fact_Checking", split=lang_arr[0].capitalize()
        )
        dataset_bool = True

if not dataset_bool:
    raise Exception("Language not supported.")

# check the input model is compatible
compatible_model_prefixes = [
    "flan",
    "t5",
    "pythia",
    "gpt",
    "opt",
    "llama",
    "roberta",
    "bert",
    "bloom",
]

model_supported = False
for model_prefix in compatible_model_prefixes:
    if model_prefix in args.model.lower():
        model_supported = True

if not model_supported:
    raise Exception("Model not supported.")

# create a config for running the pipeline
config = {
    "models": [args.model],
    "input_information": dataset,
    "verbosity": False,
}

# run the contrastive knowledge assessment function
# logs saved at './content/logging/'
score_dicts = compare_models(
    config["models"], config["input_information"], config["verbosity"]
)

# print the summary results
print(f"\nScore dict summary:\n{score_dicts[1]}")

In [ ]:
# colab specific code for saving logs to Drive after running
# change to path in your own drive

os.chdir("../../../")
log = glob.glob("/content/logging/*json")[0]
log_name = log.split("/")[-1]
log_new_path = f"/content/drive/MyDrive/Colab Files/cka_benchmark_logs/{log_name}"
!cp {log} '{log_new_path}'